# VEP pilot PacBio sequencing 
This example shows how to process PacBio circular consensus sequencing of a PacBio run containing VEP sequences from several different VEPs. 

Here we only analyzed two VEPs and use only a snippet of the full data set of circular consensus sequences so that the example is small and fast.

## Set up for analysis
Import necessary Python modules:

In [ ]:
import os
import tempfile
import warnings

import Bio.SeqIO

import pandas as pd

import pysam

import alignparse.ccs
import alignparse.minimap2
import alignparse.targets
import alignparse.cs_tag

Suppress warnings that clutter output:

In [ ]:
warnings.simplefilter('ignore')

Directory for output:

In [ ]:
outdir = './output_files/'
os.makedirs(outdir, exist_ok=True)

## Target amplicon
We have performed sequencing of several VEP amplicons that include VEP sequences along with a PacBio index and several other features.
The amplicons are defined as Genbank files.
First, let's just look at the files:

In [ ]:
wt_targetfile = 'input_files/LASV_G1959_WT.gb'
opt_targetfile = 'input_files/LASV_G1959_OPT.gb'


for targetfile in [wt_targetfile, opt_targetfile]:
    with open(targetfile) as f:
        print(f.read())

Read the amplicons into a `Targets` object, specifying the features that we require the target to contain:

In [ ]:
targets = alignparse.targets.Targets(seqsfile=[wt_targetfile, opt_targetfile],
                  req_features=['termini5', 'gene', 'spacer', 'index',
                                'termini3', 'variant_tag5', 'variant_tag3'],
                  allow_extra_features=True)

Plot the targets:

In [ ]:
_ = targets.plot(ax_width=10)

We can write them to a file for alignment:

## PacBio CCSs
We will align PacBio circular consensus sequences (CCSs) to the target.
First, we want to look at the CCSs.
A FASTQ file with these CCSs along with an associated report file were generated using the PacBio `ccs` program (see [here](https://github.com/PacificBiosciences/ccs) for details on `ccs`) using commands like the following (generates report file and BAM of CCSs):

    ccs --minLength 50 --maxLength 5000 \
        --minPasses 3  --minPredictedAccuracy 0.999 \
        --reportFile vep_pilot_report.txt \
        --polish --numThreads 16 \
        vep_pilot_subreads.bam vep_pilot_ccs.bam
        
The BAM file was then converted to a FASTQ file using [samtools](http://www.htslib.org/) with flags to retain the number of passes (`np`) and read quality (`rq`) flags:

    samtools bam2fq -T np,rq vep_pilot_ccs.bam > vep_pilot_ccs.fastq
    
Here is a data frame with the resulting FASTQ and BAM files:

In [ ]:
run_names = ['vep_pilot']
ccs_dir = 'input_files'

pacbio_runs = pd.DataFrame(
            {'name': run_names,
             'report': [f"{ccs_dir}/{name}_report.txt" for name in run_names],
             'fastq': [f"{ccs_dir}/{name}_ccs.fastq" for name in run_names]
             })

pacbio_runs

We create a `Summaries` object for these CCSs:

In [ ]:
ccs_summaries = alignparse.ccs.Summaries(pacbio_runs)

Plot how many ZMWs yielded CCSs:

In [ ]:
p = ccs_summaries.plot_zmw_stats()
_ = p.draw()

Statistics on the CCSs (length, number of subread passes, quality):

In [ ]:
for stat in ['length', 'passes', 'quality']:
    p = ccs_summaries.plot_ccs_stats(stat)
    _ = p.draw()

## Align CCSs to target
Now we use `minimap2` to align the CCSs to the target.

First, we create a `Mapper` object to run `minimap2`, using the options for codon-level deep mutational scanning:

In [ ]:
mapper = alignparse.minimap2.Mapper(
            options=alignparse.minimap2.OPTIONS_CODON_DMS)

print(f"Using `minimap2` {mapper.version} with these options:\n{' '.join(mapper.options)}")

Now use this mapper to do the alignments to a SAM file.
First, add the names of the desired alignment files to our data frame:

In [ ]:
pacbio_runs = pacbio_runs.assign(alignments=lambda x: outdir + x['name'] + '_alignments.sam')

pacbio_runs

Now use the mapper to actually align the FASTQ queries to the target:

In [ ]:
for tup in pacbio_runs.itertuples(index=False):
    print(f"Aligning {tup.fastq} to create {tup.alignments}...")
    targets.align(queryfile=tup.fastq,
                  alignmentfile=tup.alignments,
                  mapper=mapper)

These SAM files now contain the alignments along with the [`cs` tag](https://github.com/lh3/minimap2#cs), which contains details on the mutations:

In [ ]:
for fname in pacbio_runs['alignments'][:1]:
    with pysam.AlignmentFile(fname) as f:
        a = next(f)
        print(f"First alignment in {fname} has following `cs` tag:\n\n{a.get_tag('cs')}\n\n"
         f"and the following full line:\n\n{a.to_string()}"
         )

In [ ]:
start = int(targets.get_target('LASV_G1959_WT').get_feature('gene').start)
end = int(targets.get_target('LASV_G1959_WT').get_feature('gene').end)
# start = 12
# end = 34


for fname in pacbio_runs['alignments'][:1]:
    with pysam.AlignmentFile(fname) as f:
        for a in f:
            if a.is_unmapped:
                continue  # cannot currently handle unmapped
            vep_alignment = alignparse.cs_tag.Alignment(a)
            print(vep_alignment.cs)
            print(vep_alignment._cs_ops_ends)
            feat_cs = vep_alignment.extract_cs(start, end)
            print(feat_cs)
            feat_cs_len = 0
            if feat_cs is not None:
                for cs in alignparse.cs_tag.split_cs(feat_cs[0]):
                    feat_cs_len += alignparse.cs_tag.cs_op_len_target(cs)
                assert((feat_cs_len + feat_cs[1] + feat_cs[2]) == (end - start))
